<a href="https://colab.research.google.com/github/max2000777/Traitement-de-la-langue-naturelle/blob/main/MIDS_NLP_TP2_Knn_with_matrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document classification using the K-NN algorithm and tensorial operations

**Copiez ce notebook et mettez votre nom dans le nom de la copie**

A rendre par mail marie.candito@gmail.com, avant le **9 février minuit**, avec comme objet du mail :  **MIDS TP2**


The aim of this lab is to implement the K-NN algorithm using **tensors**, resulting in **faster** computation of the similarities (or distances) between a document to classify and all the labeled documents.

## Dataset download

The « reuters21578 » collection is a famous set of documents in English, associated to 0, 1 or n classes (« topics »). We provide a sub-set of this corpus, in which we made sure to have a single gold class for each document (to perform monolabel multiclass classification):

More precisely:
- train = medium.train.examples = 2000 documents with their gold class
- dev = medium.dev.examples 	= 200 documents with gold class
o	which you will use to test and evaluate the algorithm

- Document representation: Documents are represented using BOW vectors, in which the numbers of occurrences word forms have been divided by the total number of tokens in the document .
- Format: But NB, the provided *.examples files only contain the components with non-null values, identified using the word form itself instead of an integer position in the vector space.
- Vocabulary: the total vocabulary is the union of all features (words) in all training documents.
- "Unknown words": when applying the K-NN to an input document, its features (words) that are not in the training documents will just be ignored

Two smaller train and test files are provided in order to test your program more rapidly while programming.


In [1]:
import os, sys
import numpy as np
import plotly.express as px

if not os.path.exists('./reuters-examples/'):
  !pip install wget
  import wget

  # The URL for the dataset tar file
  url = 'http://www.linguist.univ-paris-diderot.fr/~mcandito/divers/reuters-examples.zip'


  if not os.path.exists('./reuteurs-examples.zip'):
    print('Downloading dataset')
    wget.download(url, './reuters-examples.zip')
    !unzip ./reuters-examples.zip
if not os.path.exists('./reuters-examples/small.dev.examples'):
  print("pb")


## Loading the dataset

In [2]:
class Examples:
    """
    a batch of examples:
    One example is
    - a BOW vector represented as a python dictionary, for features with non-null values only
    - a gold class

    dict_vectors = list of dictionary BOW vector
    gold_classes = list of gold classes
    """
    def __init__(self):
        self.gold_classes = []
        self.dict_vectors = []

def read_examples(infile):
    """ Reads a .examples file and returns an Examples instance.
    """

    stream = open(infile)
    examples = Examples()
    dict_vector = None
    while 1:
        line = stream.readline()
        if not line:
            break
        line = line[0:-1]
        if line.startswith("EXAMPLE_NB"):
            if dict_vector != None:
                examples.dict_vectors.append(dict_vector)
            dict_vector = {}
            cols = line.split('\t')
            gold_class = cols[3]
            examples.gold_classes.append(gold_class)
        elif line:# and dict_vector != None:
            (wordform, val) = line.split('\t')
            dict_vector[wordform] = float(val)

    if dict_vector != None:
        examples.dict_vectors.append(dict_vector)
    return examples


In [3]:
train_examples = read_examples('./reuters-examples/medium.train.examples')
dev_examples = read_examples('./reuters-examples/medium.dev.examples')

#train_examples = read_examples('./reuters-examples/small.train.examples')
#dev_examples = read_examples("./reuters-examples/small.dev.examples")



In [4]:
for element in train_examples.dict_vectors[0]:
  print(element)

limited
now
currently
over
all
trade
ended
still
its
weekly
buyers
zone
also
temporao
seems
late
only
going
under
total
weeks
has
oct
harvesting
good
march
around
means
routine
early
certificates
bean
hands
not
exporters
continued
areas
february
cruzados
held
january
arrivals
humidity
experiencing
went
aug
ports
bahia
view
old
liquor
some
crop
kilos
offer
export
are
year
dlrs
delivered
for
farmers
open
review
since
per
shipment
new
reluctant
available
be
ends
after
doubt
sold
york
although
convertible
come
by
stage
dry
on
prospects
last
fit
carnival
against
times
april
figures
rose
sept
nearby
named
smith
obtaining
period
cocoa
improving
shippers
throughout
quality
destinations
standing
would
their
there
been
few
much
estimated
brazilian
expected
hundred
was
final
midday
butter
selling
that
season
thousand
booked
sales
part
doubts
prices
july
with
argentina
again
made
this
mln
fob
will
estimates
were
cake
making
dec
published
tonne
showers
uruguay
almost
is
comissaria
it
an
currency
as

## TODO 1: build word <=> indices correspondance

In [5]:
#TODO
#fonctions du TP1
def get_vocab(documents):
    w2id = {}
    id2w = []
    id=0
    for ligne in documents:
      for token in ligne:
        if str(token) not in w2id:
          w2id[str(token)]=id;
          id+=1;
          id2w.append(str(token))
    return (w2id, id2w)

w2i,i2w = get_vocab(train_examples.dict_vectors)
print(w2i)
print(f"Vocabulary size : {len(i2w)}")


{'limited': 0, 'now': 1, 'currently': 2, 'over': 3, 'all': 4, 'trade': 5, 'ended': 6, 'still': 7, 'its': 8, 'weekly': 9, 'buyers': 10, 'zone': 11, 'also': 12, 'temporao': 13, 'seems': 14, 'late': 15, 'only': 16, 'going': 17, 'under': 18, 'total': 19, 'weeks': 20, 'has': 21, 'oct': 22, 'harvesting': 23, 'good': 24, 'march': 25, 'around': 26, 'means': 27, 'routine': 28, 'early': 29, 'certificates': 30, 'bean': 31, 'hands': 32, 'not': 33, 'exporters': 34, 'continued': 35, 'areas': 36, 'february': 37, 'cruzados': 38, 'held': 39, 'january': 40, 'arrivals': 41, 'humidity': 42, 'experiencing': 43, 'went': 44, 'aug': 45, 'ports': 46, 'bahia': 47, 'view': 48, 'old': 49, 'liquor': 50, 'some': 51, 'crop': 52, 'kilos': 53, 'offer': 54, 'export': 55, 'are': 56, 'year': 57, 'dlrs': 58, 'delivered': 59, 'for': 60, 'farmers': 61, 'open': 62, 'review': 63, 'since': 64, 'per': 65, 'shipment': 66, 'new': 67, 'reluctant': 68, 'available': 69, 'be': 70, 'ends': 71, 'after': 72, 'doubt': 73, 'sold': 74, 'yo

## TODO2 : Turn training set and test set into matrices

Represent the T=2000 training examples [vector, gold_class] using two structures:
-	X_train: a T x V matrix (ndarray), whose i-th row is the vector for the i-th example
-	Y_train: a vector of size T, for the gold classes of the T examples (actually it can be a plain python list for now).

Similarly, represent the dev data as X_dev and Y_dev.

In [6]:
np.zeros((3,2))

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [7]:
def build_matrices(examples, w2i):
  """ examples = instance of Examples
      w2i : word to index dictionary
  """
#   TODO
# Tips : start by building an X matrix with desired shape, filled with zeros,
#        and while looping over the examples, you can assign the non-null cells
  X = np.zeros((len(examples.dict_vectors),len(w2i)))
  for i in range(0,len(examples.dict_vectors))  :
    for mot in examples.dict_vectors[i] :
      if mot in w2i:
        X[i,w2i[mot]]=examples.dict_vectors[i][mot]
  Y = np.array(examples.gold_classes)
  return (X, Y)

# Organize the data into two matrices for document vectors
#                   and two lists for the gold classes
(X_train, Y_train) = build_matrices(train_examples, w2i)
(X_dev, Y_dev) = build_matrices(dev_examples, w2i)
print(f"Training matrix has shape {X_train.shape}")
print(f" Testing matrix has shape {X_dev.shape}")


Training matrix has shape (2000, 8659)
 Testing matrix has shape (200, 8659)


## TODO3 : K-NN classification:
- add the relevant methods to the K-NN class below to perform K-NN classification and its evaluation.
- **CONSTRAINT** : use matrix operations to compute the cosines of each row in X_dev with each row in X_train
  - shape will be (nb_test, nb_train)
  - no loop over rows nor columns!
- predict a class for each of the dev examples, using the train examples and K-NN algorithm, using **cosine** to identify the neighbors.
- **Evaluation**: Compute and print the resulting accuracy (percentage of dev examples for which the predicted class is the correct one)
- The optimal value of K is unknown, all you can do is test some k values and choose the best one (cf. methodology in machine learning : this is “tuning the hyperparameters”)


In [8]:
def Majoritee(M,Y,K):
    ClassToId={}
    IdToClass=[]
    c=0
    Maj=np.zeros((M.shape[0],1))
    LastPredic=['']*M.shape[0]
    scores=[0]
    for k in range(0,K):
        score=0
        for ligne in range(0,M.shape[0]):
            # si la classe est nouvelle on rajoute une colonne de 0
            if M[ligne,k] not in ClassToId:
                ClassToId[M[ligne,k]]=c
                IdToClass.append(M[ligne,k])
                c+=1
                Maj=np.hstack((Maj, np.zeros((M.shape[0], 1))))
            
            #on augmente le conteur 
            Maj[ligne,ClassToId[M[ligne,k]]]=Maj[ligne,ClassToId[M[ligne,k]]]+1
            
            #on calcul le nouveau majoriatire
            LastPredic[ligne]=IdToClass[np.argmax(Maj[ligne])]
            if LastPredic[ligne]==Y[ligne]:
                score+=1
        #calcul du score du k
        print(f"Pour k={k+1}, on a bien prédit {score} classes sur {M.shape[0]}")
        scores.append(score/M.shape[0])
    return scores, LastPredic
                


class KNN:
    """
    K-NN for document classification (multiclass)

    members =

    X_train = matrix of training example vectors
    Y_train = list of corresponding gold classes

    K = maximum number of neighbors to consider

    """
    def __init__(self, X, Y, K=1, verbose=False):
        self.X_train = X   # (nbexamples, d)
        self.Y_train = Y   # list of corresponding gold classes

        # nb neighbors to consider
        self.K = K

        self.verbose = verbose
    def predict_and_evaluate_on_set(self,X,Y):
        Anorm=self.X_train/np.linalg.norm(self.X_train,axis=1).reshape(self.X_train.shape[0],1)
        Bnorm=X/np.linalg.norm(X,axis=1).reshape(X.shape[0],1)
        MatriceCos=(Bnorm @Anorm.T)
        sorted=np.argsort(-MatriceCos, axis=1)
        return Majoritee(self.Y_train[sorted],Y,self.K)




In [11]:
# NB of NEIGHBORS

K=5

verbose = True

myclassifier = KNN(X = X_train,
                   Y = Y_train,
                   K = K,
                   verbose=verbose)

print("Evaluating on dev set...")
accuracies,Lastpredict = myclassifier.predict_and_evaluate_on_set(X_dev, Y_dev)
print(accuracies)
fig=px.scatter(accuracies)
fig.update_layout(
    xaxis_title='valeur de k',
    yaxis_title='Score',
    showlegend=False
)
fig.show()


Evaluating on dev set...
Pour k=1, on a bien prédit 154 classes sur 200
Pour k=2, on a bien prédit 154 classes sur 200
Pour k=3, on a bien prédit 155 classes sur 200
Pour k=4, on a bien prédit 167 classes sur 200
Pour k=5, on a bien prédit 162 classes sur 200
[0, 0.77, 0.77, 0.775, 0.835, 0.81]


## Efficiency tips:

If needed, improve your algorithm:
- don't recompute the training norms for all test example
- identify which steps are shared for all possible K values as opposed to steps that depend on K.
  - you could make your K-NN work for all values from k=1 to k=K, sharing steps not depending on k, and output a list of K accuracies, for k=1, k=2 .... k=K


6	Expected results
When using the small corpus (train / dev), here is the expected cos_matrix and the accuracies for a few k values:

Matrix of cosine similarities (rows = test, columns = train):

[[0.1864542  0.30658331 0.25518441 0.05080005 0.01133659]

 [0.         0.009787   0.01176674 0.70624201 0.66805963]

 [0.18749253 0.08808303 0.24710158 0.22135944 0.        ]]

ACCURACY FOR K =  1 = 100.00 (3 / 3) (use_weight = False)

ACCURACY FOR K =  2 = 66.67 (2 / 3) (use_weight = False)

ACCURACY FOR K =  3 = 66.67 (2 / 3) (use_weight = False)


On medium, here are the expected results for the first k values:
ACCURACY FOR K =  1 = 78.50 (157 / 200) (use_weight = False)

ACCURACY FOR K =  2 = 76.00 (152 / 200) (use_weight = False)

ACCURACY FOR K =  3 = 77.50 (155 / 200) (use_weight = False)

ACCURACY FOR K =  4 = 81.00 (162 / 200) (use_weight = False)

ACCURACY FOR K =  5 = 79.50 (159 / 200) (use_weight = False)

**NB**: It is ok even if you have not exactly the same results, but if accuracy is close to these values. Results may vary a little depending on the prediction in case of ties.
